<a href="https://colab.research.google.com/github/santiagoprado12/Titanic---Machine-Learning-from-Disaster/blob/main/Proyecto_Final_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("ggplot2")
install.packages("Amelia")
install.packages("caret")
install.packages("e1071")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(ggplot2)
library(Amelia)
library(caret)
library(e1071)

**Subir archivos**

In [ ]:
train<-read.csv("train.csv",stringsAsFactors = FALSE)
test<-read.csv("test.csv",stringsAsFactors = FALSE)
str(train)
str(test)

**Proporcion de los sobrevivientes**

In [ ]:
table(train$Survived)
prop.table(table(train$Survived))

**Primera prediccion (todos mueren)**

In [ ]:
test$Survived<- rep(0,418)

submit<-data.frame(PassengerId=test$PassengerId,Survived=test$Survived)
write.csv(submit, file = "allDead.csv", row.names = FALSE)

**Porcentaje de supervivencia entre hombres y mujeres**

In [ ]:
prop.table(table(train$Survived,train$Sex),2)

Del total de los hombres murió el 81.1% y de las mujeres murió el 25.8%

**Prediccion 2: hombres mueren, mujeres viven**

In [ ]:
test$Survived<- rep(0,418)
test$Survived[test$Sex=="female"]<-1
submit<-data.frame(PassengerId=test$PassengerId,Survived=test$Survived)
write.csv(submit, file = "allDead.csv", row.names = FALSE)

**Mujeres y niños primero!!**

In [ ]:
train$Age[is.na(train["Age"])]=mean(train$Age[!is.na(train["Age"])])
summary(train$Age)

train$Child<-0
train$Child[train$Age<18]<-1
train$Child <- factor(train$Child)

test$Child <- 0
test$Child[test$Age<18] <- 1
test$Child <- factor(test$Child)

table(train$Survived[train$Child==1])
aggregate(Survived ~ Child + Sex, data=train, FUN= function(x) {sum(x)/length(x)})
#prop.table(table(train$Survived[train$Child==1],train$Sex[train$Child==1]),2)

**Teniendo en cuenta el costo del tiquete y la clase**

In [ ]:

train$FareBin[train$Fare < 10] <- '<10'
train$FareBin[train$Fare >= 10 & train$Fare<20] <- '10-20'
train$FareBin[train$Fare >= 20] <- '20+'
table(train$FareBin)
aggregate(Survived ~ FareBin + Sex + Pclass, data=train, FUN= function(x) {sum(x)/length(x)})

test$FareBin<- rep(0,418)
test$FareBin[test$Fare < 10] <- '<10'
test$FareBin[test$Fare >= 10 & test$Fare<20] <- '10-20'
test$FareBin[test$Fare >= 20] <- '20+'
test$Survived<- rep(0,418)
test$Survived[test$Sex=="female"]<-1
test$Survived[test$Pclass==3 & test$FareBin=='20+']<-0
submit<-data.frame(PassengerId=test$PassengerId,Survived=test$Survived)
write.csv(submit, file = "allDead.csv", row.names = FALSE)

**Teniendo en cuenta el titulo de la persona**

In [ ]:
 train$Title<-0
 
train$Title<-sapply(train$PassengerId, FUN=function(x){strsplit(train$Name, split='[,.]')[[x]][2]})
train$Title<-sub(" ","", train$Title)
train$Title[train$Title=="Mlle"]<-"Miss"
 train$Title[train$Title=="Mme"]<-"Mrs"
train$Title[train$Title=="Lady"]<-"Miss" 
train$Title[train$Title=="Capt"]<-"Master"
train$Title[train$Title=="Don"]<-"Mr"
train$Title[train$Title=="Dr"]<-"Mr"
train$Title[train$Title=="Ms"]<-"Miss"  
train$Title[train$Title=="Jonkheer"]<-"Mr"  
train$Title[train$Title=="Sir"]<-"Mr" 
train$Title[train$Title=="Major"]<-"Master"  
train$Title[train$Title=="Col"]<-"Master"  
train$Title[train$Title=="Rev"]<-"Master"
train$Title[train$Title=="the Countess"]<-"Mrs"
table(train$Title)



In [ ]:
 test$Title<-0
 
test$Title<-sapply(1:418, FUN=function(x){strsplit(test$Name, split='[,.]')[[x]][2]})
test$Title<-sub(" ","", test$Title)
test$Title[test$Title=="Mlle"]<-"Miss"
test$Title[test$Title=="Mme"]<-"Mrs"
test$Title[test$Title=="Lady"]<-"Miss" 
test$Title[test$Title=="Capt"]<-"Master"
test$Title[test$Title=="Don"]<-"Mr"
test$Title[test$Title=="Dr"]<-"Mr"
test$Title[test$Title=="Ms"]<-"Miss"  
test$Title[test$Title=="Jonkheer"]<-"Mr"  
test$Title[test$Title=="Sir"]<-"Mr" 
test$Title[test$Title=="Major"]<-"Master"  
test$Title[test$Title=="Col"]<-"Master"  
test$Title[test$Title=="Rev"]<-"Master"
test$Title[test$Title=="the Countess"]<-"Mrs"
test$Title[test$Title=="Dona"]<-"Mrs"
table(test$Title)

**Teniendo en cuenta el tamaño de la familia**


In [ ]:
 train$FamilySize<-0
 train$FamilySize<-train$SibSp+train$Parch+1
 test$FamilySize<-0
 test$FamilySize<-test$SibSp+test$Parch+1

print("female: ")
prop.table(table(train$FamilySize[train$Sex=='female'],train$Survived[train$Sex=='female']),1)
print('male: ')
prop.table(table(train$FamilySize[train$Sex=='male'],train$Survived[train$Sex=='male']),1)
ggplot(train, aes(x =FamilySize, fill = factor(Survived)))+ geom_bar(stat='count', position='dodge')+
scale_x_continuous(breaks=c(1:max(train$FamilySize)))+labs(x = 'Textoetiqueta') + facet_grid(.~Sex)+ theme_minimal()

In [ ]:
train$Family<-0
#familia grande (mas de 4)=0
#familia pequeña (menos de 4)=1
train$Family[train$FamilySize<=4]<-1

In [ ]:
missmap(train)
missmap(test)
sum(!complete.cases(train$Age))
sum(!complete.cases(test$Age))
sum(!complete.cases(test$Fare))

In [ ]:
train$Age[is.na(train$Age)]<-mean(train$Age[!is.na(train$Age)])
missmap(train)
test$Age[is.na(test$Age)]<-mean(test$Age[!is.na(test$Age)])
test$Fare[is.na(test$Fare)]<-mean(test$Fare[!is.na(test$Fare)])
missmap(test)

In [ ]:
head(train)

In [ ]:
install.packages("randomForest")
install.packages("rpart")
install.packages("nnet")
install.packages("klaR")
install.packages("C50")
install.packages("kernlab")
install.packages("dplyr")

In [ ]:
library(randomForest)
library(rpart)
library(klaR)
library(C50)
library(nnet)
library(kernlab)
library(dplyr)

In [ ]:
train$Survived<-factor(train$Survived)
set.seed(288) #Semilla
train_control <- trainControl(method="cv",number = 10) #Cross-Validation
RF <- train(Survived ~ Title + FamilySize + Pclass + Sex + Age + SibSp + Parch 
+ Fare + Embarked, data=train, trControl = train_control, method="rf")
RF #Mostramos el modelo entrenado

In [ ]:
importance <- varImp(RF, scale=FALSE)
 plot(importance)

In [ ]:
Prediction <- predict(RF, test)
submit <- data.frame(PassengerId = test$PassengerId, Survived = Prediction)
write.csv(submit, file = "RandomForest.csv", row.names = FALSE)

In [ ]:

set.seed(288) #Semilla
train_control <- trainControl(method="cv",number = 10) #Cross-Validation
Rnet <- train(Survived ~ Title + FamilySize + Pclass + Sex + Age + SibSp + Parch 
+ Fare + Embarked, data=train, method="nnet",trControl = train_control)
Rnet #Mostramos el modelo entrenado

In [ ]:
Prediction <- predict(Rnet, test)
submit <- data.frame(PassengerId = test$PassengerId, Survived = Prediction)
write.csv(submit, file = "Neural Network", row.names = FALSE)

In [ ]:
set.seed(288) #Semilla
train_control <- trainControl(method="cv",number = 10) #Cross-Validation
Rknn <- train(Survived ~ Title + FamilySize + Pclass + Sex + Age + SibSp + Parch 
+ Fare + Embarked, data=train, method="knn",trControl = train_control)
Rknn #Mostramos el modelo entrenado

In [ ]:
Prediction <- predict(Rknn, test)
submit <- data.frame(PassengerId = test$PassengerId, Survived = Prediction)
write.csv(submit, file = "knn", row.names = FALSE)

In [ ]:
set.seed(288) #Semilla
train_control <- trainControl(method="cv",number = 10) #Cross-Validation
RC50 <- train(Survived ~ Title + FamilySize + Pclass + Sex + Age + SibSp + Parch 
+ Fare + Embarked, data=train, method="C5.0",trControl = train_control)
RC50 #Mostramos el modelo entrenado

In [ ]:
Prediction <- predict(RC50, test)
submit <- data.frame(PassengerId = test$PassengerId, Survived = Prediction)
write.csv(submit, file = "C50", row.names = FALSE)

In [ ]:
set.seed(288) #Semilla
train_control <- trainControl(method="cv",number = 10) #Cross-Validation
Rsvm <- train(Survived ~ Title + FamilySize + Pclass + Sex + Age + SibSp + Parch 
+ Fare + Embarked, data=train, method="svmLinear",trControl = train_control)
Rsvm #Mostramos el modelo entrenado

In [ ]:
Prediction <- predict(Rsvm, test)
submit <- data.frame(PassengerId = test$PassengerId, Survived = Prediction)
write.csv(submit, file = "SVM", row.names = FALSE)